# Timer

In [1]:
from time import sleep
from datetime import datetime as dt

start_time = "14.02.2023, 15:00"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'vol_q_high': [0.75], # np.arange(0.1, 1, 0.05),
                            'vol_q_low': [0.1], # np.arange(0.1, 1, 0.05), 
                            'window_low_bound': [2], 
                            'window_high_bound': [7],
                            'first_candle': [0.8], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.5], # np.arange(0.5, 1, 0.05)
                            },
                'Trend': {
                            'timeperiod': [16], # np.arange(2, 50, 2),
                            'low_bound': [0]
                            }
             }


opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is sell
Number of combinations is 12


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:50<00:00,  9.18s/it]


# Check local statistics

In [2]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
60,Pattern_Trend,0.75,0.1,1,5,0.80,0.50,6,0,83,1.598042,-1.461250,49.637458,-121.283750,1.6284,1.7386,1.5856,1.7366,1.7145,1.5962,1.5759,1.6572,1.6432,1.5754,1.6184,1.6718,1.6367,1.5842,1.6111,1.6683,1.5755,1.5418,1.4436,1.4619,1.4960,1.5279,1.5546,1.5096,-0.06,-0.14,-0.23,-0.33,-0.44,-0.55,-0.68,-0.83,-0.96,-1.12,-1.28,-1.45,-1.62,-1.73,-1.84,-1.96,-2.08,-2.20,-2.31,-2.43,-2.55,-2.66,-2.76,-2.86
61,Pattern_Trend,0.75,0.1,1,5,0.80,0.55,6,0,77,1.650387,-1.356667,50.079837,-104.463333,1.6325,1.8382,1.7095,1.8691,1.8156,1.6808,1.6410,1.7292,1.7053,1.6275,1.6724,1.7287,1.6877,1.6285,1.6562,1.7161,1.6132,1.5741,1.4681,1.4791,1.5117,1.5388,1.5670,1.5190,-0.05,-0.11,-0.18,-0.27,-0.36,-0.47,-0.59,-0.72,-0.85,-1.00,-1.15,-1.32,-1.49,-1.59,-1.71,-1.82,-1.95,-2.07,-2.18,-2.30,-2.42,-2.55,-2.65,-2.76
62,Pattern_Trend,0.75,0.1,1,5,0.80,0.60,6,0,71,1.671062,-1.353750,47.645437,-96.116250,1.4638,1.6746,1.5982,1.7779,1.7477,1.6534,1.6076,1.6981,1.6729,1.5997,1.6476,1.6838,1.6575,1.6691,1.7111,1.7695,1.7969,1.7434,1.6027,1.6132,1.6511,1.6834,1.7212,1.6611,-0.04,-0.10,-0.16,-0.25,-0.35,-0.45,-0.57,-0.72,-0.84,-1.00,-1.16,-1.32,-1.49,-1.59,-1.71,-1.82,-1.95,-2.07,-2.18,-2.30,-2.43,-2.55,-2.66,-2.78
70,Pattern_Trend,0.75,0.1,1,5,0.85,0.50,6,0,67,1.654892,-1.342917,43.877742,-89.975417,1.2641,1.6904,1.5678,1.7098,1.7151,1.5690,1.5462,1.6526,1.6213,1.5636,1.6442,1.7254,1.6886,1.6322,1.6999,1.8181,1.7000,1.6774,1.6308,1.6551,1.6997,1.7431,1.7835,1.7195,-0.06,-0.13,-0.20,-0.28,-0.36,-0.47,-0.59,-0.74,-0.86,-0.99,-1.14,-1.30,-1.47,-1.57,-1.69,-1.80,-1.92,-2.04,-2.15,-2.26,-2.38,-2.50,-2.61,-2.72
72,Pattern_Trend,0.75,0.1,1,5,0.85,0.60,6,0,57,1.873287,-1.318333,49.777387,-75.145000,1.0852,1.7040,1.6771,1.8589,1.8738,1.7350,1.6673,1.7844,1.7309,1.6641,1.7576,1.8346,1.8143,1.8481,1.9510,2.0916,2.1543,2.1075,2.0256,2.0446,2.0958,2.1444,2.2080,2.1008,-0.04,-0.09,-0.14,-0.22,-0.29,-0.39,-0.52,-0.67,-0.80,-0.94,-1.09,-1.25,-1.44,-1.54,-1.66,-1.78,-1.91,-2.04,-2.15,-2.27,-2.40,-2.54,-2.67,-2.80
50,Pattern_Trend,0.75,0.1,1,5,0.75,0.50,6,0,93,1.573783,-1.317500,53.361850,-122.527500,1.5884,1.7562,1.6726,1.7992,1.7497,1.6559,1.6225,1.6148,1.5905,1.5385,1.5257,1.5040,1.4828,1.4995,1.5297,1.5911,1.5319,1.5040,1.4429,1.4573,1.4862,1.5294,1.5657,1.5323,-0.06,-0.13,-0.22,-0.32,-0.41,-0.52,-0.64,-0.77,-0.89,-1.03,-1.17,-1.32,-1.46,-1.55,-1.65,-1.75,-1.86,-1.96,-2.06,-2.17,-2.27,-2.38,-2.47,-2.56
35,Pattern_Trend,0.75,0.1,1,5,0.65,0.75,6,0,78,1.298104,-1.316667,23.252125,-102.700000,1.2878,1.3029,1.2442,1.2692,1.2945,1.3079,1.2723,1.3069,1.2851,1.2860,1.2839,1.2491,1.2368,1.2546,1.2717,1.3244,1.3736,1.3495,1.2942,1.2863,1.3307,1.3548,1.3698,1.3183,-0.07,-0.14,-0.24,-0.35,-0.46,-0.58,-0.59,-0.73,-0.84,-0.99,-1.14,-1.28,-1.42,-1.58,-1.67,-1.77,-1.87,-1.96,-2.05,-2.15,-2.27,-2.38,-2.49,-2.58
15,Pattern_Trend,0.75,0.1,1,5,0.55,0.75,6,0,85,1.400683,-1.298333,34.058083,-110.358333,1.4383,1.4101,1.3630,1.3897,1.4243,1.4257,1.3828,1.4080,1.3774,1.3749,1.3684,1.3428,1.3384,1.3617,1.3785,1.4195,1.4660,1.4409,1.3852,1.3758,1.4258,1.4509,1.4611,1.4072,-0.06,-0.12,-0.22,-0.32,-0.43,-0.55,-0.5

In [4]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
23,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,48,0,62,1.434696,-1.525833,26.951142,-94.601667,1.5319,1.6625,1.6120,1.8184,1.5913,1.4930,1.5399,1.5224,1.4063,1.3090,1.4626,1.4015,1.4202,1.3758,1.3805,1.4082,1.3222,1.3033,1.3189,1.3085,1.3023,1.3039,1.3217,1.3164,-0.09,-0.18,-0.29,-0.40,-0.51,-0.64,-0.78,-0.92,-1.07,-1.19,-1.34,-1.51,-1.69,-1.80,-1.92,-2.03,-2.14,-2.26,-2.36,-2.48,-2.60,-2.70,-2.84,-2.88
19,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,40,0,62,1.356046,-1.450833,22.074842,-89.951667,1.6608,1.8224,1.7534,1.9474,1.6633,1.5140,1.5603,1.5385,1.4161,1.3178,1.2538,1.2054,1.2181,1.1823,1.1888,1.2228,1.1532,1.1374,1.1331,1.1267,1.1236,1.1269,1.1586,1.1204,-0.09,-0.19,-0.30,-0.41,-0.52,-0.65,-0.79,-0.93,-1.07,-1.20,-1.33,-1.49,-1.65,-1.75,-1.85,-1.94,-2.03,-2.12,-2.21,-2.30,-2.38,-2.46,-2.57,-2.59
21,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,44,0,61,1.328346,-1.447083,20.029096,-88.272083,1.6065,1.7755,1.7138,1.9002,1.6253,1.4828,1.5302,1.5120,1.3917,1.2956,1.2286,1.1792,1.1930,1.1582,1.1641,1.1958,1.1287,1.1134,1.1096,1.1034,1.1007,1.1041,1.1349,1.1330,-0.09,-0.19,-0.30,-0.42,-0.53,-0.65,-0.80,-0.94,-1.09,-1.21,-1.34,-1.50,-1.66,-1.76,-1.85,-1.94,-2.02,-2.11,-2.19,-2.27,-2.36,-2.43,-2.53,-2.55
20,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,42,0,61,1.328346,-1.447083,20.029096,-88.272083,1.6065,1.7755,1.7138,1.9002,1.6253,1.4828,1.5302,1.5120,1.3917,1.2956,1.2286,1.1792,1.1930,1.1582,1.1641,1.1958,1.1287,1.1134,1.1096,1.1034,1.1007,1.1041,1.1349,1.1330,-0.09,-0.19,-0.30,-0.42,-0.53,-0.65,-0.80,-0.94,-1.09,-1.21,-1.34,-1.50,-1.66,-1.76,-1.85,-1.94,-2.02,-2.11,-2.19,-2.27,-2.36,-2.43,-2.53,-2.55
22,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,46,0,61,1.325275,-1.446250,19.841775,-88.221250,1.6161,1.7204,1.6578,1.8472,1.6063,1.4784,1.5255,1.5080,1.3891,1.2939,1.2275,1.1823,1.2080,1.1725,1.1782,1.2099,1.1410,1.1254,1.1213,1.1130,1.1101,1.1130,1.1323,1.1294,-0.09,-0.19,-0.30,-0.42,-0.52,-0.65,-0.80,-0.94,-1.09,-1.21,-1.34,-1.49,-1.66,-1.75,-1.85,-1.94,-2.03,-2.11,-2.19,-2.27,-2.36,-2.43,-2.53,-2.55
18,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,38,0,64,1.389138,-1.442500,24.904800,-92.320000,1.7101,1.8360,1.8028,1.9950,1.7103,1.5559,1.6082,1.5809,1.4562,1.3553,1.2885,1.2372,1.2484,1.2116,1.2182,1.2513,1.1796,1.1635,1.1585,1.1520,1.1485,1.1516,1.1805,1.1392,-0.09,-0.19,-0.31,-0.43,-0.53,-0.66,-0.80,-0.94,-1.08,-1.20,-1.32,-1.48,-1.64,-1.74,-1.83,-1.92,-2.01,-2.10,-2.19,-2.27,-2.36,-2.43,-2.54,-2.56
16,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,34,0,67,1.341158,-1.391667,22.857608,-93.241667,1.7302,1.8485,1.8166,2.0506,1.6939,1.5238,1.5466,1.4912,1.3833,1.2671,1.2144,1.1536,1.1668,1.1368,1.1437,1.1865,1.1222,1.1085,1.1053,1.0997,1.0973,1.1004,1.1188,1.0820,-0.10,-0.20,-0.31,-0.43,-0.54,-0.66,-0.79,-0.92,-1.06,-1.17,-1.29,-1.44,-1.60,-1.69,-1.77,-1.86,-1.94,-2.02,-2.09,-2.17,-2.24,-2.31,-2.39,-2.41
17,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,36,0,65,1.322750,-1.387083,20.978750,-90.160417,1.7429,1.8562,1.8203,2.0082,1.6398,1.4842,1.5179,1.4763,1.3686,1.2507,1.1982,1.1377,1.1518,1.1217,1.1290,1.1611,1.1022,1.0886,1.0857,1.0802,1.0780,1.0812,1.1003,1.0652,-0.10,-0.20,-0.31,-0.44,-0.54,-0.66,-0.80,-0.93,-1.06

In [9]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
270,Pattern_Trend,0.85,0.10,1,5,0.8,0.5,6,0,73,1.485546,-1.515833,35.444846,-110.655833,1.4915,1.6405,1.5783,1.6914,1.6736,1.5652,1.5411,1.5468,1.5198,1.4805,1.4537,1.4073,1.4118,1.3772,1.4047,1.4810,1.4167,1.4018,1.3679,1.3885,1.4245,1.4575,1.4872,1.4446,-0.07,-0.16,-0.26,-0.37,-0.48,-0.60,-0.74,-0.89,-1.03,-1.20,-1.36,-1.52,-1.69,-1.79,-1.90,-2.02,-2.13,-2.25,-2.36,-2.48,-2.61,-2.72,-2.82,-2.93
252,Pattern_Trend,0.80,0.10,1,5,0.8,0.5,6,0,80,1.579912,-1.483750,46.393000,-118.700000,1.7607,1.8546,1.7064,1.8034,1.7671,1.6492,1.6385,1.6306,1.5987,1.5577,1.5324,1.4990,1.4895,1.4499,1.4760,1.5553,1.4888,1.4736,1.4343,1.4548,1.4931,1.5246,1.5585,1.5212,-0.07,-0.15,-0.25,-0.35,-0.46,-0.58,-0.72,-0.87,-1.00,-1.16,-1.32,-1.48,-1.64,-1.75,-1.86,-1.98,-2.09,-2.20,-2.31,-2.44,-2.56,-2.68,-2.79,-2.90
234,Pattern_Trend,0.75,0.10,1,5,0.8,0.5,6,0,87,1.490817,-1.360000,42.701050,-118.320000,1.6460,1.7377,1.5863,1.7035,1.6757,1.5791,1.5733,1.5759,1.5497,1.4932,1.4748,1.4442,1.4247,1.3890,1.4164,1.4699,1.4035,1.3779,1.3214,1.3383,1.3730,1.4011,1.4300,1.3950,-0.06,-0.13,-0.21,-0.31,-0.41,-0.52,-0.64,-0.78,-0.90,-1.05,-1.20,-1.35,-1.50,-1.60,-1.71,-1.82,-1.93,-2.04,-2.14,-2.25,-2.37,-2.48,-2.57,-2.67
306,Pattern_Trend,0.95,0.10,1,5,0.8,0.5,6,0,59,1.504987,-1.282500,29.794262,-75.667500,1.6488,1.9316,1.7246,1.8283,1.7783,1.6136,1.5782,1.6000,1.5589,1.5033,1.4453,1.3960,1.3901,1.3427,1.3745,1.4578,1.3801,1.3619,1.2994,1.3221,1.3667,1.4030,1.4327,1.3818,-0.06,-0.14,-0.23,-0.34,-0.44,-0.55,-0.66,-0.79,-0.90,-1.03,-1.17,-1.31,-1.47,-1.54,-1.63,-1.73,-1.81,-1.90,-1.98,-2.06,-2.14,-2.22,-2.30,-2.38
288,Pattern_Trend,0.90,0.10,1,5,0.8,0.5,6,0,67,1.433192,-1.258333,29.023842,-84.308333,1.6029,1.6884,1.5650,1.6671,1.6328,1.5200,1.4951,1.5044,1.4776,1.4334,1.3819,1.3394,1.3321,1.2941,1.3246,1.4021,1.3341,1.3177,1.2864,1.3077,1.3449,1.3787,1.4064,1.3598,-0.06,-0.15,-0.24,-0.34,-0.43,-0.53,-0.63,-0.75,-0.86,-1.00,-1.15,-1.29,-1.44,-1.52,-1.61,-1.69,-1.77,-1.86,-1.94,-2.03,-2.11,-2.19,-2.27,-2.34
271,Pattern_Trend,0.85,0.15,1,5,0.8,0.5,6,0,111,1.362883,-1.197083,40.280050,-132.876250,1.2585,1.2980,1.4303,1.3802,1.4038,1.3200,1.3275,1.3518,1.3661,1.3512,1.3377,1.3375,1.3309,1.3026,1.3651,1.4309,1.3864,1.3991,1.3756,1.3782,1.3835,1.3879,1.4196,1.3868,-0.05,-0.11,-0.19,-0.29,-0.37,-0.47,-0.57,-0.68,-0.79,-0.92,-1.05,-1.18,-1.31,-1.38,-1.48,-1.59,-1.70,-1.80,-1.90,-2.01,-2.10,-2.18,-2.26,-2.35
253,Pattern_Trend,0.80,0.15,1,5,0.8,0.5,6,0,118,1.424275,-1.194167,50.064450,-140.911667,1.4320,1.4309,1.5120,1.4523,1.4663,1.3766,1.3904,1.4075,1.4182,1.4021,1.3895,1.3973,1.3826,1.3512,1.4119,1.4794,1.4337,1.4457,1.4187,1.4213,1.4285,1.4320,1.4660,1.4365,-0.05,-0.11,-0.19,-0.28,-0.36,-0.46,-0.56,-0.68,-0.78,-0.91,-1.04,-1.17,-1.30,-1.38,-1.48,-1.59,-1.69,-1.80,-1.90,-2.01,-2.10,-2.19,-2.27,-2.36
180,Pattern_Trend,0.60,0.10,1,5,0.8,0.5,6,0,109,1.407208,-1.191667,44.385708,-129.891667,1.4277,1.5028,1.3534,1.4605,1.4447,1.4115,1.4337,1.4578,1.4460,1.3742,1.3622,1.3525,1.3328,1.3086,1.3757,1.4225,1.3793,1.3939,1.3535,1.3717,1.4019,1.4643,1.4859,1.4559,-0.05,-0.10,-0.16,-0.24,-0.33,-0.43,

In [2]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
9,Pattern_Trend,0.75,0.1,3,5,0.8,0.5,16,0,59,1.287129,-1.517500,16.940621,-89.532500,1.7546,2.0386,1.8253,1.8354,1.6273,1.4334,1.3912,1.3721,1.2775,1.1856,1.1474,1.0788,1.0837,1.0605,1.0819,1.1270,1.0726,1.0625,1.0601,1.0613,1.0722,1.0856,1.0776,1.0789,-0.11,-0.23,-0.36,-0.50,-0.63,-0.77,-0.91,-1.08,-1.19,-1.33,-1.46,-1.61,-1.78,-1.76,-1.84,-1.92,-2.02,-2.12,-2.21,-2.32,-2.43,-2.53,-2.64,-2.67
5,Pattern_Trend,0.75,0.1,2,5,0.8,0.5,16,0,77,1.402475,-1.368750,30.990575,-105.393750,1.6707,2.0247,1.8614,2.0044,1.8137,1.5672,1.5197,1.4980,1.4091,1.3185,1.2839,1.2237,1.2120,1.1907,1.2182,1.2644,1.1987,1.1906,1.1857,1.1864,1.2066,1.2225,1.2110,1.1776,-0.10,-0.21,-0.32,-0.44,-0.55,-0.67,-0.79,-0.93,-1.03,-1.14,-1.27,-1.41,-1.56,-1.57,-1.66,-1.75,-1.84,-1.94,-2.04,-2.14,-2.25,-2.33,-2.44,-2.47
11,Pattern_Trend,0.75,0.1,3,7,0.8,0.5,16,0,78,1.454137,-1.312917,35.422725,-102.407500,1.8561,1.9832,1.8854,2.0649,1.8910,1.6761,1.6168,1.5376,1.4600,1.3784,1.3462,1.2864,1.2745,1.2519,1.2719,1.2918,1.2368,1.2308,1.2208,1.2213,1.2296,1.2405,1.2243,1.2230,-0.09,-0.19,-0.25,-0.38,-0.49,-0.62,-0.74,-0.87,-0.97,-1.09,-1.21,-1.34,-1.49,-1.50,-1.59,-1.67,-1.76,-1.86,-1.97,-2.08,-2.20,-2.30,-2.40,-2.45
1,Pattern_Trend,0.75,0.1,1,5,0.8,0.5,16,0,80,1.390475,-1.261667,31.238000,-100.933333,1.6615,2.0028,1.8537,2.0367,1.7876,1.6054,1.5660,1.5049,1.3974,1.2897,1.2554,1.2011,1.1878,1.1684,1.1948,1.2427,1.1816,1.1713,1.1657,1.1664,1.1857,1.1967,1.1904,1.1577,-0.10,-0.19,-0.30,-0.41,-0.52,-0.63,-0.75,-0.87,-0.97,-1.07,-1.18,-1.31,-1.45,-1.45,-1.53,-1.61,-1.69,-1.78,-1.86,-1.96,-2.05,-2.13,-2.22,-2.25
7,Pattern_Trend,0.75,0.1,2,7,0.8,0.5,16,0,96,1.530229,-1.230833,50.902000,-118.160000,1.7749,1.9810,1.9045,2.1698,2.0082,1.7521,1.6909,1.6195,1.5471,1.4663,1.4372,1.3862,1.3594,1.3391,1.3642,1.3870,1.3246,1.3201,1.3092,1.3095,1.3259,1.3394,1.3209,1.2885,-0.08,-0.18,-0.24,-0.35,-0.46,-0.56,-0.67,-0.79,-0.88,-0.99,-1.10,-1.23,-1.37,-1.39,-1.49,-1.58,-1.67,-1.77,-1.88,-1.98,-2.09,-2.18,-2.28,-2.33
10,Pattern_Trend,0.75,0.1,3,6,0.8,0.5,16,0,69,1.311638,-1.221667,21.502987,-84.295000,1.6574,1.8048,1.7235,1.9064,1.7564,1.5228,1.4666,1.4036,1.3241,1.2421,1.1968,1.1329,1.1344,1.1124,1.1350,1.1561,1.1063,1.0971,1.0894,1.0903,1.0997,1.1113,1.1056,1.1043,-0.08,-0.18,-0.22,-0.35,-0.46,-0.57,-0.69,-0.81,-0.90,-1.02,-1.13,-1.26,-1.42,-1.40,-1.48,-1.56,-1.64,-1.73,-1.83,-1.93,-2.03,-2.13,-2.23,-2.27
6,Pattern_Trend,0.75,0.1,2,6,0.8,0.5,16,0,87,1.410754,-1.148750,35.735613,-99.941250,1.6022,1.8303,1.7691,2.0415,1.9021,1.6260,1.5669,1.5075,1.4325,1.3514,1.3103,1.2544,1.2422,1.2222,1.2501,1.2738,1.2146,1.2073,1.1979,1.1985,1.2164,1.2305,1.2213,1.1891,-0.08,-0.17,-0.22,-0.33,-0.42,-0.52,-0.62,-0.73,-0.82,-0.92,-1.02,-1.15,-1.30,-1.30,-1.39,-1.47,-1.56,-1.66,-1.75,-1.85,-1.95,-2.04,-2.13,-2.17
3,Pattern_Trend,0.75,0.1,1,7,0.8,0.5,16,0,99,1.515483,-1.147917,51.032850,-113.643750,1.7651,1.9663,1.8974,2.1915,1.9835,1.7799,1.7257,1.6214,1.5318,1.4350,1.4055,1.3595,1.3325,1.3139,1.3382,1.3637,1.3051,1.2985,1.2871,1.2875,1.3033,1.3128,1.2992,1.2672,-0.08,-0.17,-0.22,-0.33,-0.43,-0.54,-0.64,-0.75,-0.84,

# Research parameter influence

In [8]:
param = 'Pattern_second_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_second_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,0.854897,-0.304701,-168.318122,-286.847375,68130
1,0.6,0.850164,-0.288188,-150.008786,-258.243681,60370
2,0.7,0.849519,-0.281743,-133.779457,-224.315035,52007
3,0.8,0.839842,-0.299785,-109.302578,-205.237062,41617
4,0.9,0.871600,-0.322160,-72.804460,-170.303528,30776


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {
                            'vol_q_high': [0.75],
                            'vol_q_low': [0.1],
                            'window_low_bound': [2], 
                            'window_high_bound': [7],
                            'first_candle': [0.8],
                            'second_candle': [0.5],
                            },
                'Trend': {
                            'timeperiod': [16],
                            'low_bound': [0]
                            }
             }
 
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)